In [ ]:
import tensorflow as tf
import numpy as np
import pickle
import random
import os
from sklearn.metrics import roc_curve

class LogisticRegression(tf.keras.Model):
    def __init__(self, config):
        super(LogisticRegression, self).__init__()
        self.optimizer = tf.keras.optimizers.Adam(config["learning_rate"])

        self.concatenation = tf.keras.layers.Concatenate(axis=1, name="concatenation")
        self.lr = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid, name="lr",
        kernel_regularizer=tf.keras.regularizers.L2(l2=config["l2_reg"]))

    def call(self, x, d):
        x = unit_normalization(x)
        return self.lr(self.concatenation([x, d]))

def compute_loss(model, x, d, label):
    prediction = model(x, d)
    loss_sum = tf.negative(tf.add(tf.multiply(5, tf.multiply(label, tf.math.log(prediction))), 
                                  tf.multiply(tf.subtract(1., label), tf.math.log(tf.subtract(1., prediction)))))
    return tf.reduce_mean(loss_sum)

def calculate_auc(model, test_x, test_d, test_y, config):
    AUC = tf.keras.metrics.AUC(num_thresholds=200)
    AUC.reset_states()
    x, d, y = pad_matrix(test_x, test_d, test_y, config)
    pred = model(x, d)
    AUC.update_state(y, pred)

    return AUC.result().numpy()

def calculate_ROC(model, test_x, test_d, test_y, config):
    x, d, y = pad_matrix(test_x, test_d, test_y, config)
    pred = model(x,d)
    fpr, tpr, thresholds = roc_curve(test_y, pred)
    return fpr, tpr, thresholds

def calculate_ppv(model, test_x, test_d, test_y, config):
    ppv = tf.keras.metrics.Precision(thresholds=0.8)
    ppv.reset_states()
    x, d, y = pad_matrix(test_x, test_d, test_y, config)
    pred = model(x,d)
    ppv.update_state(y, pred)
    return ppv.result().numpy()

def load_data(patient_record_path, demo_record_path, labels_path):
    patient_record = pickle.load(open(patient_record_path, 'rb'))
    demo_record = pickle.load(open(demo_record_path, 'rb'))
    labels = pickle.load(open(labels_path, 'rb'))
    
    return patient_record, demo_record, labels

def save_data(output_path, mydata):
    with open(output_path, 'wb') as f:
        pickle.dump(mydata, f)

def pad_matrix(records, demos, labels, config):
    n_patients = len(records)
    input_vocabsize = config["input_vocabsize"]
    demo_vocabsize = config["demo_vocabsize"]

    x = np.zeros((n_patients, input_vocabsize)).astype(np.float32) # sum of all visits of the patient
    d = np.zeros((n_patients, demo_vocabsize)).astype(np.float32)
    y = np.array(labels).astype(np.float32)
    
    for idx, rec in enumerate(records):
        for visit in rec:
            x[idx, visit] += 1

    x = np.clip(0, 1, x) # clip values bigger than 1.
        
    for idx, demo in enumerate(demos):
        d[idx, int(demo[:-2])] = 1. # the last element of demos is age 
        d[idx, -1:] = demo[-1:]
        
    return x, d, y

def shuffle_data(data1, data2, data3):
    data1, data2, data3 = np.array(data1), np.array(data2), np.array(data3)
    idx = np.arange(len(data1))
    random.seed(1234)
    random.shuffle(idx)

    return data1[idx], data2[idx], data3[idx]

def unit_normalization(myarray):
    avg = tf.reshape(tf.math.reduce_mean(myarray, axis=-1), shape=(myarray.shape[0], 1))
    std = tf.reshape(tf.math.reduce_std(myarray, axis=-1), shape=(myarray.shape[0], 1))
    return tf.math.divide(tf.math.subtract(myarray, avg), std)

In [ ]:
def train_lreg_kfold(output_path, patient_record_path, demo_record_path, labels_path, epochs, batch_size,
                input_vocabsize, demo_vocabsize, l2_reg=0.001, learning_rate=0.001, k=5, times =5, notes=None):
    
    tf.random.set_seed(1234)
    config = locals().copy()
    for i in range(times):
        version = i
        print("load data...")
        recs, demos, labels = load_data(patient_record_path, demo_record_path, labels_path)

        print("split the dataset into k-fold...")
        recs, demos, labels = shuffle_data(recs, demos, labels)
        chunk_size = int(np.floor(len(labels) / k))
        np.split(np.arange(len(labels)), [chunk_size*i for i in range(k)])
        folds = np.tile(np.split(np.arange(len(labels)), [chunk_size*i for i in range(int(k))])[1:], 2)

        k_fold_auc = []
        k_fold_ppv = []
        k_fold_tpr = []
        mean_fpr = np.linspace(0,1,200)
        k_fold_training_loss = []

        for i in range(k):
            train_x, test_x = recs[np.concatenate(folds[(i%k):(i%k)+k-1])], recs[folds[(i%k)+k]]
            train_d, test_d = demos[np.concatenate(folds[(i%k):(i%k)+k-1])], demos[folds[(i%k)+k]]
            train_y, test_y = labels[np.concatenate(folds[(i%k):(i%k)+k-1])], labels[folds[(i%k)+k]]

            num_batches = int(np.ceil(float(len(train_x)) / float(batch_size)))
            training_loss = []

            print("build and initialize model for {k}th fold...".format(k=i+1))
            lr_model = LogisticRegression(config)

            best_auc = 0
            best_epoch = 0
            best_model = None
            print("start training...")
            for epoch in range(epochs):
                loss_record = []
                progbar = tf.keras.utils.Progbar(num_batches)

                for t in random.sample(range(num_batches), num_batches):
                    batch_x = train_x[t * batch_size:(t+1) * batch_size]
                    batch_d = train_d[t * batch_size:(t+1) * batch_size]
                    batch_y = train_y[t * batch_size:(t+1) * batch_size]

                    x, d, y = pad_matrix(batch_x, batch_d, batch_y, config)

                    with tf.GradientTape() as tape:
                        batch_cost = compute_loss(lr_model, x, d, y)
                    gradients = tape.gradient(batch_cost, lr_model.trainable_variables)
                    lr_model.optimizer.apply_gradients(zip(gradients, lr_model.trainable_variables))

                    loss_record.append(batch_cost.numpy())
                    progbar.add(1)

                print('epoch:{e}, mean loss:{l:.6f}'.format(e=epoch+1, l=np.mean(loss_record)))
                training_loss.append(np.mean(loss_record))
                current_auc = calculate_auc(lr_model, test_x, test_d, test_y, config)
                #print('epoch:{e}, current auc:{l:.6f}'.format(e=epoch+1, l=current_auc))
                if current_auc > best_auc: 
                    best_auc = current_auc
                    best_epoch = epoch+1
                    best_model = lr_model.get_weights()

            k_fold_training_loss.append(training_loss)
            print("calculate AUC on the best model using the test set")
            lr_model.set_weights(best_model)
            test_auc = calculate_auc(lr_model, test_x, test_d, test_y, config)
            test_ppv = calculate_ppv(lr_model, test_x, test_d, test_y, config)
            print("AUC of {k}th fold: {auc:.6f}".format(k=i+1, auc=test_auc))
            print("PPV of {k}th fold: {ppv:.6f}".format(k=i+1, ppv=test_ppv))
            k_fold_auc.append(test_auc)
            k_fold_ppv.append(test_ppv)
            fpr, tpr, thresholds = calculate_ROC(lr_model, test_x, test_d, test_y, config)
            k_fold_tpr.append(np.interp(mean_fpr, fpr, tpr))

        print("saving k-fold results...")
        mode_name = "mhot"
        #np.save(os.path.join(output_path, "LRS_{m}_{k}fold_l{l}_training_loss_ver{i}.npy".format(k=k, m=mode_name, l=learning_rate, i=version)), k_fold_training_loss)
        np.save(os.path.join(output_path, "LRS_{m}_{k}fold_l{l}_auc_ver{i}.npy".format(k=k, m=mode_name, l=learning_rate, i=version)), k_fold_auc)
        np.save(os.path.join(output_path, "LRS_{m}_{k}fold_l{l}_tpr_ver{i}.npy".format(k=k, m=mode_name, l=learning_rate, i=version)), k_fold_tpr)
        np.save(os.path.join(output_path, "LRS_{m}_{k}fold_l{l}_ppv_ver{i}.npy".format(k=k, m=mode_name, l=learning_rate, i=version)), k_fold_ppv)
        np.save(os.path.join(output_path, "LRS_{m}_e{e}_l{l}_model_ver{i}.npy".format(m=mode_name, e=epochs, l=learning_rate, i=version)), lr_model.get_weights())

    save_data(os.path.join(output_path, "LRS_{m}_{k}fold_l{l}_config.pkl".format(k=k, m=mode_name, l=learning_rate)), config)

In [ ]:
train_lreg_kfold('./ml_output', './patient_record.pkl', './demo_record.pkl','./labels.pkl', 20, 2,
                1318, 4, l2_reg=0.001, learning_rate=0.001, k=5, times=5)  